In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Apr 19 23:18:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              43W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
import scipy.ndimage
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.metrics import MeanSquaredError
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%load_ext tensorboard

In [ ]:
import matplotlib.pyplot as plt
def augment_3d(image, label):
    # Randomly flip the image along the axial plane
    if np.random.rand() > 0.5:
        image = np.flip(image, axis=0)

    # Randomly flip the image along the coronal plane
    if np.random.rand() > 0.5:
        image = np.flip(image, axis=1)

    # Randomly rotate the image
    angles = [0, 90, 180, 270]
    angle = np.random.choice(angles)
    image = scipy.ndimage.rotate(image, angle, axes=(0, 1), reshape=False, mode='nearest')

    return image, label
def augment_3d_and_display(image):
    fig, axes = plt.subplots(1, 5, figsize=(20, 4))

    # Original Image
    axes[0].imshow(image[:, :, image.shape[2] // 2], cmap='gray')
    axes[0].set_title('Original')

    # Axial Flip
    axial_flip = np.flip(image, axis=0)
    axes[1].imshow(axial_flip[:, :, axial_flip.shape[2] // 2], cmap='gray')
    axes[1].set_title('Axial Flip')

    # Coronal Flip
    coronal_flip = np.flip(image, axis=1)
    axes[2].imshow(coronal_flip[:, :, coronal_flip.shape[2] // 2], cmap='gray')
    axes[2].set_title('Coronal Flip')

    # 90 Degree Rotation
    rotation_90 = scipy.ndimage.rotate(image, 90, axes=(0, 1), reshape=False)
    axes[3].imshow(rotation_90[:, :, rotation_90.shape[2] // 2], cmap='gray')
    axes[3].set_title('90 Degree Rotation')

    # 180 Degree Rotation
    rotation_180 = scipy.ndimage.rotate(image, 180, axes=(0, 1), reshape=False)
    axes[4].imshow(rotation_180[:, :, rotation_180.shape[2] // 2], cmap='gray')
    axes[4].set_title('180 Degree Rotation')

    plt.tight_layout()
    plt.show()


def data_generator(X, y, batch_size=4, augment=False):

    dataset_size = X.shape[0]
    indices = np.arange(dataset_size)
    np.random.shuffle(indices)

    while True:
        for start_idx in range(0, dataset_size, batch_size):
            end_idx = min(start_idx + batch_size, dataset_size)
            batch_indices = indices[start_idx:end_idx]

            X_batch = np.empty((len(batch_indices),) + X.shape[1:], dtype=np.float32)
            y_batch = y[batch_indices]

            for i, idx in enumerate(batch_indices):
                image, label = X[idx], y_batch[i]
                if augment:
                    image, label = augment_3d(image, label)
                X_batch[i] = image

            yield X_batch, y_batch

assert tf.__version__.startswith('2.')



def create_folder_if_not_exists(folder_path): #If a folder doesn't exist, create it
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created.")
    else:
        print(f"Folder '{folder_path}' already exists.")



def load_nii(file_path): #load .nii images
    img = nib.load(file_path).get_fdata()
    img = np.expand_dims(img, axis=-1)
    return img

def load_data_and_labels(data_dir, labels_file): #load all the images and images
    labels_df = pd.read_excel(labels_file)
    labels_dict = dict(zip(labels_df.iloc[:, 0], labels_df.iloc[:, 1:].values))

    data = []
    labels = []
    for filename in os.listdir(data_dir):
        if filename.endswith('.nii'):
            file_path = os.path.join(data_dir, filename)
            img = load_nii(file_path)
            key = filename.replace("smwp1","smwp1_").replace("_T1.nii","")
            if key in labels_dict:
                data.append(img)
                labels.append(labels_dict[key])

    print("Data Loaded")
    return np.array(data), np.array(labels)

def normalize_data(input_data, output_data): #min-max normalization
    print(np.min(input_data))
    print(np.max(input_data))
    print(np.min(output_data))
    print(np.max(output_data))
    input_norm = (input_data - np.min(input_data)) / (np.max(input_data) - np.min(input_data))
    output_norm = (output_data - np.min(output_data)) / (np.max(output_data) - np.min(output_data))
    print("Data Normalized")
    return input_norm, output_norm

def preprocess_images(images, target_shape=(128, 144, 128)): #padded data for uniformity


    pad_height = (target_shape[0] - images.shape[1]) / 2
    pad_width = (target_shape[1] - images.shape[2]) / 2
    pad_depth = (target_shape[2] - images.shape[3]) / 2

    padding = (
        (0, 0),
        (int(np.floor(pad_height)), int(np.ceil(pad_height))),
        (int(np.floor(pad_width)), int(np.ceil(pad_width))),
        (int(np.floor(pad_depth)), int(np.ceil(pad_depth))),
        (0, 0)
    )

    preprocessed_images = np.pad(images, padding, mode='constant', constant_values=0)

    return preprocessed_images

def resnet3d_block(input_tensor, filters, kernel_size=(3, 3, 3), strides=(1, 1, 1)):

    x = layers.Conv3D(filters, kernel_size, padding='same', strides=strides)(input_tensor) #conv3d
    x = layers.BatchNormalization()(x) #bn
    x = layers.Activation('relu')(x) #relu

    x = layers.Conv3D(filters, kernel_size, padding='same')(x) #conv3d
    x = layers.BatchNormalization()(x) #bn

    x = layers.add([x, input_tensor]) #add previous copy of input to current features through skip connection

    return layers.Activation('relu')(x) #relu on the added input+current features again

def build_3d_resnet(input_shape=(128, 128, 128, 1), num_classes=1):
    inputs = layers.Input(shape=input_shape) #Input Layer Initialized

    x = layers.Conv3D(64, (7, 7, 7), strides=(2, 2, 2), padding='same')(inputs) #Conv3D on the input with 64 filters, stride=2 and keeping the padding same so size of input and output are same
    x = layers.BatchNormalization()(x) #Conv3d output batch normalized mean output close to 0 and the output standard deviation close to 1, regularized
    x = layers.Activation('relu')(x) #non-linearity added between input and output with non-negative values
    x = layers.MaxPooling3D((3, 3, 3), strides=(2, 2, 2), padding='same')(x) #downsamples x along with dim reduction along each channel with a stride of 2 taking max value over each 3*3*3 voxel and padding same to keep more info at edges


    x = resnet3d_block(x, 64)
    x = resnet3d_block(x, 64) #two blocks of resnet for simplication


    x = layers.GlobalAveragePooling3D()(x)  #avg of feature map to a single scalar value - (32, 36, 32, 64) to (,64)
    outputs = layers.Dense(num_classes, activation='linear')(x) #Dense Layer with 1 output (the predicted regression value) - (,64) to (,1)
    model = models.Model(inputs, outputs)
    return model


def root_mean_squared_error(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

def mean_squared_error(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_pred - y_true))

def load_and_preprocess_data(data_dir, labels_file):
    data, labels = load_data_and_labels(data_dir, labels_file) #load data and labels into data,labels from the directory
    data, labels = normalize_data(data, labels) #normalize the data
    data = preprocess_images(data, target_shape=(128, 144, 128)) #reshape data by padding it and adding uniformity
    print("Data Loaded and Preprocessed. Data shape:", data.shape)

    X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.20, random_state=42) #split data into train and val in 80:20 ratio

    return X_train, X_val, y_train, y_val

def main():
    #Initialize all the folders
    data_dir = '/content/drive/MyDrive/Data Mining 8650/n171_smwp1/'
    labels_file = '/content/drive/MyDrive/Data Mining 8650/PTs_500_4k_blinded.xlsx'
    tensorboard_log_dir = '/content/drive/MyDrive/Data Mining 8650/logs'

    create_folder_if_not_exists(tensorboard_log_dir)
    print("Directory Called")


    X_train, X_val, y_train, y_val = load_and_preprocess_data(data_dir, labels_file) #Preprocess the data

    #first_image = X_train[0]
    #augment_3d_and_display(first_image)

    model = build_3d_resnet(input_shape=X_train.shape[1:], num_classes=1) #build the resnet model with height,width,depth,channels of the images with 1 output (predicted threshold)

    model.summary() #show a summary of the layers, output shape, # of Parameters

    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=[mean_squared_error, root_mean_squared_error])

    tensorboard_cb = TensorBoard(log_dir=tensorboard_log_dir)
    model_checkpoint_cb = ModelCheckpoint(filepath='model_checkpoint.h5', save_best_only=True, monitor='val_loss', mode='min')


    train_gen = data_generator(X_train, y_train, batch_size=4, augment=True)
    val_gen = data_generator(X_val, y_val, batch_size=4)

    train_steps_per_epoch = np.ceil(len(X_train) / 4).astype(int)
    val_steps_per_epoch = np.ceil(len(X_val) / 4).astype(int)

    model.fit(train_gen, steps_per_epoch=train_steps_per_epoch, validation_data=val_gen, validation_steps=val_steps_per_epoch, epochs=1000, callbacks=[tensorboard_cb, model_checkpoint_cb])


if __name__ == "__main__":
    main()

In [ ]:
%tensorboard --logdir "/content/drive/MyDrive/Data Mining 8650/logs"

import IPython

display(IPython.display.HTML('''
<button id='open_tb'>Open TensorBoard</button>
<button id='hide_tb'>Hide TensorBoard</button>
<script>document.querySelector('#open_tb').onclick = () => { window.open(document.querySelector('iframe').src, "__blank") }
        document.querySelector('#hide_tb').onclick = () => { document.querySelector('iframe').style.display = "none" }</script>'''))